Criar uma tabela de ordens ja com:
quantidade de itens, vendedores, lojas, mapeamento de Status e valor total da venda (valor total - Desconto)

Mapeamento Status:
  1 THEN "Pending"
  2 THEN "Processing"
  3 THEN "Shipped"
  4 THEN "Delivered"
  ELSE "Unknown"

In [0]:
%python
bronze_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/silver/"
fold_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/gold/"
resource_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/resource/origem"
resource_path_volume = "/Volumes/bikestore/logistica/bikestore_resource/origem/"

In [0]:
%python
bronze_map= {
    "tmp_bronze_brands": f"{bronze_path}/brands/",
    "tmp_bronze_categories": f"{bronze_path}/categories/",
    "tmp_bronze_customers": f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders": f"{bronze_path}/orders/",
    "tmp_bronze_products": f"{bronze_path}/products/",
    "tmp_bronze_staffs": f"{bronze_path}/staffs/",
    "tmp_bronze_stocks": f"{bronze_path}/stocks/",
    "tmp_bronze_stores": f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format("delta")
     .load(path)
     .createOrReplaceTempView(view_name))

In [0]:
%python
df_orders_silver = spark.sql("""
                              with order_items as (
  select
    oi.order_id,
    oi.item_id,
    oi.product_id,
    oi.quantity,
    oi.list_price,
    oi.discount,
    ROUND(oi.list_price * oi.quantity * (1 - oi.discount), 2) as total_sale
  from
    tmp_bronze_order_items oi
)
select
  o.order_id,
  o.customer_id,
  CASE
    WHEN o.order_status = 1 THEN "Pending"
    WHEN o.order_status = 2 THEN "Processing"
    WHEN o.order_status = 3 THEN "Shipped"
    WHEN o.order_status = 4 THEN "Delivered"
    ELSE "Unknown"
  END status,
  o.order_status,
  st.store_name,
  st.city,
  st.state,
  s.active,
  s.first_name,
  s.email,
  o.order_date,
  o.required_date,
  o.shipped_date,
  oi.product_id,
  oi.quantity,
  oi.total_sale,
  oi.list_price,
  oi.discount
from
  tmp_bronze_orders as o
    left join tmp_bronze_staffs as s
      on o.staff_id = s.staff_id
    left join tmp_bronze_stores as st
      on o.store_id = st.store_id
    left join order_items as oi
      on o.order_id = oi.order_id                             
                                   """)

# salvando em delta na Silver
df_orders_silver.write.mode("overwrite").format("delta").save(f"{silver_path}/orders")

In [0]:
create table if not exists bikestore.logistica.silver_orders
location "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/silver/orders"

In [0]:
SELECT * FROM bikestore.logistica.silver_orders LIMIT 15